In [1]:
import random
import numpy as np
import matplotlib.pyplot as plt
import string
import math
%matplotlib inline

In [2]:
#data storage
inputs = [[1,0],[0,0],[0,1],[1,1]]
outputs = [1,0,1,0]

In [3]:
# location of different normalizing operations
def noChange(n):
    return n

def sigmoid( x):
    return 1/(1+np.exp(-x))

def abstanh(x):
    return math.fabs(math.tanh(x))

def absolute(x):
    return math.fabs(x)

In [4]:
class Individual(object):
    def __init__(self, instructions, name):
        
        self.instructions = instructions
        self.name = name
    
    def predict(self, oneInputSet):
        nodeVals = oneInputSet[:] + [0]*(len(nodes)-len(oneInputSet))
        changedNodes = [False]*len(nodes)
        
        for instruction in self.instructions:
            
            if not changedNodes[instruction[0]]:            
                changedNodes[instruction[0]] = True
                
                #change this for different type of functions
                nodeVals[instruction[0]] = absolute(nodeVals[instruction[0]])
                
            nodeVals[instruction[1]] += nodeVals[instruction[0]] * instruction[2]
        
        return absolute(nodeVals[outputNode])
    
    def fitness(self, inputs, output):   
        store = 0
        for index in range(len(inputs)):
            store += (output[index] - self.predict(inputs[index]))**2
        self.fitVal = store**0.5
        return self.fitVal 

In [145]:
class Breed(object):
    
    def __init__(self, populationSize, inputs, outputs, mutationLevel, evolvulationLevel, retainPercentage):
        self.populationSize = populationSize
        self.retainSize = int(populationSize*retainPercentage)
        
        self.mutationLevel = mutationLevel
        self.evolvulationLevel = evolvulationLevel
        
        self.inputs = inputs
        self.outputs = outputs
        
        self.species = []
        self.generationNumber = 0
    

        self.individuals = []
        #this the conenctions can change
        
    def initializePopulation(self):        
        name = list(range(len(nodes)))
        for i in range(self.populationSize):
            #this has to  match the length of connectionsNames
            weights = [random.uniform(-1,1), random.uniform(-1,1)]
            instructions = [connections[index] + [weights[index]] for index in range(len(connections))]
            self.individuals.append(Individual(instructions, name))
    
    def evalPopulation(self):
        totalFitness = 0
        for individual in self.individuals:
            totalFitness += individual.fitness(self.inputs, self.outputs)
        self.GenerationFitness = totalFitness / self.populationSize
        return self.GenerationFitness
    
    def weedPopulation(self):
        self.oldGeneration = []
        for individual in self.individuals:
            if individual.fitVal < self.GenerationFitness:
                self.oldGeneration.append(individual)
        
        self.oldGeneration = sorted(self.oldGeneration, key = lambda cell: cell.fitVal)
        
        while len(self.oldGeneration) < self.retainSize:
            self.oldGeneration.append(self.individuals[random.randint(0,len(self.individuals)-1)])
        
        while len(self.oldGeneration)> self.retainSize:
            del self.oldGeneration[-1]
            del self.oldGeneration[random.randint(0,len(self.oldGeneration)-1)]
        
        sumed = 0
        for individual in self.oldGeneration:
            sumed += individual.fitVal
        self.oldAVG = sumed / len(self.oldGeneration)
    
    def breed(self):
        
        
        if random.random() <  self.evolvulationLevel:
            print('***************we are evolving******************')
            self.evolve()
        self.newGeneration = self.oldGeneration[:]
        self.addMutants()
        self.mate()
        self.individuals = self.newGeneration
    
    def addMutants(self):
        for individual in self.oldGeneration:
            if random.random() < self.mutationLevel:
                newInstructions = [x[:] for x in individual.instructions]
                newInstructions[random.randint(0,len(newInstructions)-1)][2] = random.uniform(-1,1)
                
                self.newGeneration.append(Individual(newInstructions, individual.name))
    #### mate needs to make sure the instructions match up
    def mate(self):
        #print("mating starting")
        while len(self.newGeneration) < self.populationSize:
            father = self.oldGeneration[random.randint(0, len(self.oldGeneration)-1)]
            mother = self.newGeneration[random.randint(0, len(self.newGeneration)-1)]
            if father is not mother:
                #print("father is not mother")
                if father.name == mother.name:
                    #print("father is same specie as mother")
                    newInstructions = [ x[:] for x in father.instructions]
                    for index in range(len(newInstructions)):
                        newInstructions[index][2] = 0
                        if random.random()>0.5:
                            
                            for instruction in father.instructions:
                                if instruction[:2] == newInstructions[index][:2]:
                                    newInstructions[index][2] = instruction[2]
                                    break
                            
                        else:
                            for instruction in mother.instructions:
                                if instruction[:2] == newInstructions[index][:2]:
                                    newInstructions[index][2] = instruction[2]
                                    break
                           
                    self.newGeneration.append(Individual(newInstructions, father.name))
 
    def checkPath(self, valid, exploringNode, path, newConnections, checkedConnections, depth, instructions, depthToInstruction):
        print('called to check on', exploringNode)
        
        if depth >= maxDepth:
            print("max depth has been reached")
            return False
        
        #depth +=1
        
        if not valid:
            print("somehow passed a not valid function")
            return False
                   
        for index in range(len(newConnections)):
            
            if newConnections[index][1] == exploringNode:
                print()
                print('current depth', depth)
                print(exploringNode, "connects to:", newConnections[index][0])
                
                newpath = path[:] + [newConnections[index][0]]
                print("current path:", newpath)
                
                if len(np.unique(newpath)) != len(newpath):
                    print('individual is no longer valid')
                    valid = False
                    return False
                
                if not checkedConnections[index]:
                    instructions.insert(depthToInstruction[depth], newConnections[index])
                    print("inserting instruction at location",depthToInstruction[depth])
                    for index1 in range(depth, maxDepth):
                        depthToInstruction[index1] += 1
                    print(depthToInstruction)
                    print(instructions)
                    checkedConnections[index] = True
                
                
             
            
                
                if nodes[newConnections[index][0]] != 'input':
                    print("recursively calling on check Path for node", newConnections[index][0])
                    newDepth = depth + 1
                    boolean = self.checkPath(valid, newConnections[index][0], newpath, newConnections, checkedConnections, newDepth, instructions, depthToInstruction)
                    #if not boolean:
                    #    return False
        return True
    
    def evolve(self):
        startNode = None
        endNode = None
        addNode = False
        name = self.oldGeneration[0].name[:]
        oldInstructions = self.oldGeneration[0].instructions[:]
        newConnections = []
        mutantConnections = []
        mutationLocation = []
        newSpecie = []
        
        print(' starting evolution')
        
        #finding the start and end Node
        while True:
            startNode = random.choice(name)
            if nodes[startNode] == 'output':
                continue
            
            endNode = random.choice(name)
            if nodes[endNode] != 'input' and endNode != startNode:
                break
            
        print(startNode, endNode)
        
        #checking if we need to add a new node
        for connection in oldInstructions:
            if [startNode, endNode] == connection[:2]:
                addNode = True
                break
        print(addNode)
        
        #creating list of conncetions
        for instruction in oldInstructions:
            newConnections.append(instruction[:2])
            
        if addNode:
            newNode = len(nodes)
            nodes.append('hidden')
            newConnections.append([startNode, newNode])
            mutantConnections.append([startNode, newNode])
            newConnections.append([newNode, endNode])
            mutantConnections.append([newNode, endNode])
            name.append(newNode)
        else:
            newConnections.append([startNode, endNode])
            mutantConnections.append([startNode, endNode])
        
        print(newConnections)
        
        #creating new instructions
        checkedConnections = [False]*len(newConnections)
        valid = True
        exploringNode = outputNode
        path = []
        depth = 0
        newInstructions = []
        depthToInstruction = [0]*maxDepth
        
        self.checkPath(valid, exploringNode, path, newConnections, checkedConnections, depth, newInstructions, depthToInstruction)
        if valid:
            newInstructions.reverse()
        else:
            self.evolve()
            return
         
        print(newInstructions)
        
        #finding mutationLocation
        for connection in mutantConnections:
            mutationLocation.append(newInstructions.index(connection))
            oldInstructions.insert( mutationLocation[-1], connection + [0]) 
        
        newInstructions = oldInstructions
        print(newInstructions)
        
        #modifying oldGeneration Instruction to new instructions
        
        #creating new Specie
        for i in range(int(populationSize * retainPercentage)):
            for index in mutationLocation:
                newInstructions[index][2] = random.uniform(-1,1)
            newSpecie.append( Individual(newInstructions, name))
        print( 'finished generating new specie' )
        
        #training new species
        mutantEvolution = Breed(len(newSpecie), inputs, outputs, mutationLevel, 0, retainPercentage*2)
        mutantEvolution.individuals = newSpecie
        for i in range(7):
            mutantEvolution.evalPopulation()
            mutantEvolution.weedPopulation()
            mutantEvolution.breed()
        self.oldGeneration.extend(mutantEvolution.individuals)
        

In [146]:
#global variables
outputNode = 2
maxDepth = 10
#nodes
nodes = []
nodes.append('input')
nodes.append('input')
nodes.append('output')

#connections
connections = []
connections.append([0,2])
connections.append([1,2])

In [147]:
populationSize = 100
mutationLevel = 0.5
evolutionLevel = 0.0
retainPercentage = 0.2
breed = Breed(populationSize, inputs, outputs, mutationLevel, evolutionLevel, retainPercentage)
breed.initializePopulation()

In [160]:
breed.evalPopulation()
breed.weedPopulation()
breed.breed()

In [161]:
breed.evalPopulation()
breed.weedPopulation()
breed.evolve()
breed.breed()

 starting evolution
0 3
False
[[3, 4], [1, 3], [4, 2], [3, 2], [0, 2], [1, 2], [0, 3]]
called to check on 2

current depth 0
2 connects to: 4
current path: [4]
inserting instruction at location 0
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[[4, 2]]
recursively calling on check Path for node 4
called to check on 4

current depth 1
4 connects to: 3
current path: [4, 3]
inserting instruction at location 1
[1, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[[4, 2], [3, 4]]
recursively calling on check Path for node 3
called to check on 3

current depth 2
3 connects to: 1
current path: [4, 3, 1]
inserting instruction at location 2
[1, 2, 3, 3, 3, 3, 3, 3, 3, 3]
[[4, 2], [3, 4], [1, 3]]

current depth 2
3 connects to: 0
current path: [4, 3, 0]
inserting instruction at location 3
[1, 2, 4, 4, 4, 4, 4, 4, 4, 4]
[[4, 2], [3, 4], [1, 3], [0, 3]]

current depth 0
2 connects to: 3
current path: [3]
inserting instruction at location 1
[2, 3, 5, 5, 5, 5, 5, 5, 5, 5]
[[4, 2], [3, 2], [3, 4], [1, 3], [0, 3]]
recursively calling on ch

In [156]:
for individual in breed.oldGeneration:
    print(individual.name)

[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]
[0, 1, 2, 3, 4]


In [157]:
for individual in breed.oldGeneration:
    print(individual.instructions, individual.fitVal)

[[1, 3, 0.7440384886522728], [3, 2, 0.1527062456826649], [0, 2, -0.9723114431526463], [1, 2, 0.8595401244166909]] 0.0385718872982812
[[1, 3, 0.7440384886522728], [3, 2, 0.1527062456826649], [0, 2, -0.9723114431526463], [1, 2, 0.8595401244166909]] 0.0385718872982812
[[1, 3, 0.7440384886522728], [3, 2, 0.1527062456826649], [0, 2, -0.9723114431526463], [1, 2, 0.8595401244166909]] 0.0385718872982812
[[1, 3, 0.7440384886522728], [3, 2, 0.1527062456826649], [0, 2, -0.9723114431526463], [1, 2, 0.8595401244166909]] 0.0385718872982812
[[1, 3, 0.7440384886522728], [3, 2, 0.1527062456826649], [0, 2, -0.9723114431526463], [1, 2, 0.8595401244166909]] 0.0385718872982812
[[1, 3, 0.7440384886522728], [3, 2, 0.1527062456826649], [0, 2, -0.9723114431526463], [1, 2, 0.8595401244166909]] 0.0385718872982812
[[1, 3, 0.7440384886522728], [3, 2, 0.1527062456826649], [0, 2, -0.9723114431526463], [1, 2, 0.8595401244166909]] 0.0385718872982812
[[1, 3, 0.7440384886522728], [3, 2, 0.1527062456826649], [0, 2, -0.97

AttributeError: 'Individual' object has no attribute 'fitVal'

In [183]:
residual = []

In [184]:
for i in range(100):
    residual.append(breed.evalPopulation())
    breed.weedPopulation()
    if i%20 is 0:
        print('************GENERATION {}**************'.format(i))
        for individual in breed.oldGeneration:
            print(individual.name)
    breed.breed()

************GENERATION 0**************
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
***************we are evolving******************
[[0, 2], [1, 2]]
startNode: 1 
 endNode: 2
[[0, 2], [1, 2], [1, 2]]
addNode isTrue
[[0, 2, -0.9675450684179734], [1, 2, 0.9803505225583975]]
[[0, 2, -0.9675450684179734], [1, 3, 0.06781513020467878], [3, 2, 0.8736530271603269], [1, 2, 0.9803505225583975]]
***************we are evolving******************
[[0, 2], [1, 2], [1, 3], [3, 2]]
startNode: 0 
 endNode: 2
[[0, 2], [1, 2], [3, 2], [0, 2]]
addNode isTrue
[[0, 2, -0.9675450684179734], [1, 2, 0.9803505225583975]]
[[0, 4, -0.3460252213314272], [4, 2, -0.01329778258342329], [0, 2, -0.9675450684179734], [1, 2, 0.9803505225583975]]
************GENERATION 20**************
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1, 2]
[0, 1

ValueError: [3, 5] is not in list

In [144]:
alist = [[1,2],[3,4],[5,6]]
try:
    for 
except Exception:
    print('oops')

SyntaxError: invalid syntax (<ipython-input-144-36a222ceac0e>, line 3)

In [ ]:
breed.evalPopulation()

In [179]:
true = True
print("hi" + str(true))

hiTrue


In [395]:
startNode = 0
endNode = 3
newConnections = connections[:] 
newConnections.append([startNode, endNode])
checkedConnections = [False]*len(newConnections)
valid = True
exploringNode = outputNode
path = []
depth = 0
instructions = []
depthToInstruction = [0]*maxDepth

print(newConnections)

breed.checkPath(valid, exploringNode, path, newConnections, checkedConnections, depth, instructions, depthToInstruction)

[[0, 2], [1, 2], [1, 3], [3, 2], [0, 3]]
called to check on 2

2 connects to: 0
current path: [0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
inserting instruction at location 1
[[0, 2]]

2 connects to: 1
current path: [1]
[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
inserting instruction at location 2
[[0, 2], [1, 2]]

2 connects to: 3
current path: [3]
[3, 3, 3, 3, 3, 3, 3, 3, 3, 3]
inserting instruction at location 3
[[0, 2], [1, 2], [3, 2]]
recursively calling on check Path for node 3
called to check on 3

3 connects to: 1
current path: [3, 1]
[3, 4, 4, 4, 4, 4, 4, 4, 4, 4]
inserting instruction at location 4
[[0, 2], [1, 2], [3, 2], [1, 3]]

3 connects to: 0
current path: [3, 0]
[3, 5, 5, 5, 5, 5, 5, 5, 5, 5]
inserting instruction at location 5
[[0, 2], [1, 2], [3, 2], [0, 3], [1, 3]]


False

In [392]:
connections.append([1,3])
connections.append([3,2])

In [393]:
nodes.append('hidden')

In [394]:
connections, nodes

([[0, 2], [1, 2], [1, 3], [3, 2]], ['input', 'input', 'output', 'hidden'])

In [163]:
for oneINput in inputs:
    print(breed.oldGeneration[0].predict(oneINput))

0.9775642243999767
0.0
0.9758040355202552
0.0017601888797215137


In [165]:
outputs

[1, 0, 1, 0]

In [166]:
inputs

[[1, 0], [0, 0], [0, 1], [1, 1]]